In [16]:
import sys
if "google.colab" in sys.modules:
    print("Colab Detected")
    import tensorflow as tf
    gpus = tf.config.list_physical_devices("GPU")
    if not gpus:
        raise RuntimeError("Nessuna GPU trovata.Controlla di aver selezionato il runtime giusto.")
    else:
        print(f"Trovate {len(gpus)} GPU:\n{gpus}")
    
    !git clone https://github.com/AtomicDonuts/Progetto_Computings.git
    %cd Progetto_Computings/
    !git checkout modell
    !pip install -q -r requirements.txt
    !python3 fits_import/fits2csv.py
    
    sys.path.append("imports/")
    import nn_models as ann
    import custom_variables as custom_paths
    import metrics as met
else:
    print("Local Machine Detected")
    sys.path.append("../imports/")
    import nn_models as ann
    import custom_variables as custom_paths
    import metrics as met

Local Machine Detected


In [17]:
import numpy as np
import pandas as pd
import keras
import keras_tuner as kt
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [18]:
df = pd.read_csv(custom_paths.csv_path)
df = df[(df["CLASS_GENERIC"] == "AGN") | (df["CLASS_GENERIC"] == "Pulsar")]
print(f"Sample Size: {len(df)}")

Sample Size: 4334


In [19]:
df["PowerLaw"] = np.where(df["SpectrumType"] == "PowerLaw",1,0,)
df["LogParabola"] = np.where(df["SpectrumType"] == "LogParabola",1,0,)
df["PLSuperExpCutoff"] = np.where(df["SpectrumType"] == "PLSuperExpCutoff",1,0,)

In [20]:
col_input1 = ["GLAT", "PowerLaw","LogParabola","PLSuperExpCutoff" ,"Variability_Index"]
col_flux_band = np.array([[f"Flux_Band_{i}", f"Sqrt_TS_Band_{i}"] for i in range(8)])
col_flux_hist = np.array([[f"Flux_History_{i}", f"Sqrt_TS_History_{i}"] for i in range(14)])

In [21]:
norm_cols = np.array(list(col_flux_band.flatten()) + list(col_flux_hist.flatten()))
scaler = StandardScaler()
scaler.fit(df[norm_cols])
scaled_data = scaler.transform(df[norm_cols])
df[norm_cols] = scaled_data

In [22]:
input_additional = df[col_input1].to_numpy()
input_flux_hist = df[col_flux_hist.flatten()].to_numpy()
input_flux_band = df[col_flux_band.flatten()].to_numpy()
print(f"Additionl Size: {input_additional.shape}")
print(f"Flux_Band Size: {input_flux_band.shape}")
print(f"Flux_History Size: {input_flux_hist.shape}")

Additionl Size: (4334, 5)
Flux_Band Size: (4334, 16)
Flux_History Size: (4334, 28)


In [23]:
is_agn = df["CLASS_GENERIC"].to_numpy() == "AGN"

labels = np.zeros((len(df)), dtype=int)
labels[~is_agn] = 1

labels_double = np.zeros((len(df), 2), dtype=int)
labels_double[is_agn, 0] = 1
labels_double[~is_agn, 1] = 1

In [24]:
from sklearn.utils import class_weight

class_weight = class_weight.compute_class_weight(
    class_weight="balanced", classes=np.unique(labels), y=labels
)
class_weight = {index: value for index, value in enumerate(class_weight)}

In [25]:
skf = StratifiedKFold(n_splits=2,shuffle=True)
train,test = next(skf.split(np.zeros(len(labels)),labels))

In [26]:
model = ann.final_model(input_flux_band.shape[1:],input_flux_hist.shape[1:],input_additional.shape[1:])

In [27]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)

history = model.fit(
    x=[input_flux_band[train], input_flux_hist[train], input_additional[train]],
    y=labels,
    batch_size=32,
    validation_split=0.5,
    epochs=300,
    class_weight=class_weight,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6945 - auc: 0.4848 - loss: 0.8549 - val_accuracy: 0.9419 - val_auc: 0.5027 - val_loss: 0.3518 - learning_rate: 0.0100
Epoch 2/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9702 - auc: 0.4799 - loss: 0.4705 - val_accuracy: 0.9419 - val_auc: 0.4946 - val_loss: 0.4327 - learning_rate: 0.0100
Epoch 3/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9821 - auc: 0.4992 - loss: 0.3388 - val_accuracy: 0.9419 - val_auc: 0.5229 - val_loss: 0.4478 - learning_rate: 0.0100
Epoch 4/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9757 - auc: 0.5727 - loss: 0.4225 - val_accuracy: 0.9419 - val_auc: 0.5065 - val_loss: 0.4249 - learning_rate: 0.0100
Epoch 5/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9773 - auc: 0.6861 - loss: 0.3624 - val_accuracy: 0.9419 - val_auc: 0.5247 - val_loss: 0.4053 - learning_rate: 0.0100
Epoch 6/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9796 - auc: 0.5

In [28]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

# =============================================================================
# 1. PREPARAZIONE DEI DATI E PREDISPOSIZIONE (Il fix per il tuo errore è qui)
# =============================================================================

# Ottieni le probabilità dalla rete neurale
# Supponiamo che X_test siano i tuoi dati di test
y_pred_probs = model.predict(
    [input_flux_band[test], input_flux_hist[test], input_additional[test]]
)
y_test = labels[test]

# GESTIONE DELLE ETICHETTE (FIX per ValueError)
# Se y_test è in formato One-Hot (es. [[1, 0], [0, 1]]), lo convertiamo in indici [0, 1]
# Classe 0 = AGN, Classe 1 = PSR
if y_test.ndim > 1 and y_test.shape[1] > 1:
    y_test_binary = np.argmax(y_test, axis=1)
else:
    y_test_binary = y_test

# GESTIONE DELLE PROBABILITÀ
# Se l'output è Softmax (N, 2), prendiamo la colonna 1 (probabilità di essere PSR)
if y_pred_probs.shape[1] == 2:
    y_scores = y_pred_probs[:, 1]
else:
    # Se l'output è Sigmoide (N, 1), lo appiattiamo
    y_scores = y_pred_probs.flatten()

# =============================================================================
# 2. RICERCA DEL DECISION BOUNDARY OTTIMALE
# =============================================================================
# Il paper suggerisce di ottimizzare la soglia per massimizzare l'accuratezza
# o per ottenere "Equal Accuracy" tra le classi[cite: 282, 285].

thresholds = np.arange(0., 1.00, 0.001)

best_acc = 0.0
best_threshold = 0.5

min_diff_equal_acc = 1.0
equal_acc_threshold = 0.5

print("Avvio ottimizzazione del decision boundary...")

for thresh in thresholds:
    # Binarizza le predizioni in base alla soglia corrente
    current_preds = (y_scores > thresh).astype(int)

    # --- A. Calcolo Best Accuracy ---
    acc = accuracy_score(y_test_binary, current_preds)
    if acc > best_acc:
        best_acc = acc
        best_threshold = thresh

    # --- B. Calcolo Equal Accuracy ---
    # Calcoliamo la matrice di confusione per questa soglia
    tn, fp, fn, tp = confusion_matrix(y_test_binary, current_preds).ravel()

    # Accuratezza sulla classe 0 (AGN) -> Specificity
    acc_class_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
    # Accuratezza sulla classe 1 (PSR) -> Sensitivity/Recall
    acc_class_1 = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Cerchiamo la soglia dove la differenza tra le due accuratezze è minima
    diff = abs(acc_class_0 - acc_class_1)
    if diff < min_diff_equal_acc:
        min_diff_equal_acc = diff
        equal_acc_threshold = thresh

print("-" * 30)
print(f"Soglia per Max Accuracy:  {best_threshold:.2f} (Acc: {best_acc:.4f})")
print(
    f"Soglia per Equal Accuracy: {equal_acc_threshold:.2f} (Diff: {min_diff_equal_acc:.4f})"
)
print("-" * 30)

# =============================================================================
# 3. VALUTAZIONE FINALE (Usando la soglia scelta)
# =============================================================================

# Scegli quale soglia usare (di solito Max Accuracy per performance generale)
final_threshold = best_threshold
y_final_pred = (y_scores > final_threshold).astype(int)

# --- Calcolo Metriche ---

# 1. Accuracy
final_acc = accuracy_score(y_test_binary, y_final_pred)

# 2. F1 Score (Importante per dataset sbilanciati come AGN >> PSR) [cite: 276]
final_f1 = f1_score(y_test_binary, y_final_pred)

# 3. ROC AUC (Indipendente dalla soglia, usa le probabilità grezze) [cite: 279]
final_auc = roc_auc_score(y_test_binary, y_scores)

# 4. Matrice di Confusione
cm = confusion_matrix(y_test_binary, y_final_pred)

print(f"\nRISULTATI FINALI (Soglia {final_threshold:.2f}):")
print(f"Accuracy: {final_acc:.4f}")
print(f"F1 Score: {final_f1:.4f}")  
print(f"ROC AUC:  {final_auc:.4f}")
#print(f"{met.class_accuracy(equal_acc_threshold,labels,y_scores)}")
print("\nMatrice di Confusione:")
print("Predicted AGN | Predicted PSR")
print(f"True AGN: {cm[0,0]}           {cm[0,1]}")
print(f"True PSR: {cm[1,0]}           {cm[1,1]}")

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Avvio ottimizzazione del decision boundary...
------------------------------
Soglia per Max Accuracy:  0.43 (Acc: 0.9262)
Soglia per Equal Accuracy: 0.20 (Diff: 0.0016)
------------------------------

RISULTATI FINALI (Soglia 0.43):
Accuracy: 0.9262
F1 Score: 0.0000
ROC AUC:  0.3325

Matrice di Confusione:
Predicted AGN | Predicted PSR
True AGN: 2007           0
True PSR: 160           0


Avvio ottimizzazione del decision boundary...
------------------------------
Soglia per Max Accuracy:  0.45 (Acc: 0.9781)
Soglia per Equal Accuracy: 0.03 (Diff: 0.0026)
------------------------------

RISULTATI FINALI (Soglia 0.45):
Accuracy: 0.9781
F1 Score: 0.8354
ROC AUC:  0.9822
(np.float64(0.9369706028898854), np.float64(0.934375))

Matrice di Confusione:
Predicted AGN | Predicted PSR
True AGN: 3998           16
True PSR: 79           241